# Step 2: Train model and artifact export

Train prediciton model and export model + preprocessor


In [0]:
import pickle

from sklearn.datasets import load_iris
import tensorflow as tf

from preprocess import MySimpleScaler

iris = load_iris()
scaler = MySimpleScaler()
num_classes = len(iris.target_names)
X = scaler.preprocess(iris.data)
y = tf.keras.utils.to_categorical(iris.target, num_classes=num_classes)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(25, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(25, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
model.compile(
  optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=1)

#export
model.save('model.h5')
with open ('preprocessor.pkl', 'wb') as f:
  pickle.dump(scaler, f)

# Step 3: Upload exported data

general name variables

In [0]:
import uuid
REGION="us-central1" #sever region of google cloud ml
PROJECT_ID = 'autodeploy1-264112' #The project you have created in google
BUCKET_NAME = "colab_deploy_v2" + str(uuid.uuid1()) #name of the google storage bucket where the data gets uploaded

In [0]:
MODEL_NAME='IrisPredictor1' #name of the deployed model
VERSION_NAME='v1' #version of the deployed model

Authentication with your account. Paste the access code

In [0]:
from google.colab import auth
auth.authenticate_user()

set the gcloud project in which we are working

In [0]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [0]:
#create bucket
!gsutil mb gs://{BUCKET_NAME}

Creating gs://colab_deploy_v2e052c34e-2fc7-11ea-8c94-0242ac1c0002/...


Zip files based on setup.py

In [0]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating my_custom_code.egg-info
writing my_custom_code.egg-info/PKG-INFO
writing dependency_links to my_custom_code.egg-info/dependency_links.txt
writing top-level names to my_custom_code.egg-info/top_level.txt
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'
reading manifest file 'my_custom_code.egg-info/SOURCES.txt'
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'

running check


creating my_custom_code-0.1
creating my_custom_code-0.1/my_custom_code.egg-info
copying files to my_custom_code-0.1...
copying predictor.py -> my_custom_code-0.1
copying preprocess.py -> my_custom_code-0.1
copying setup.py -> my_custom_code-0.1
copying my_custom_code.egg-info/PKG-INFO -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/SOURCES.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/dependency_links.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/top_lev

**AI Platform needs access to the following files to deploy custom pipeline procedure:**

*   model.h5 (exported trained model artifact)
*   preprocessor.pkl (exported preprocess artifact)
*   my_custom_code-0.1.tar.gz (custom code)






Upload the files to the created bucket

In [0]:
# source distribution package (Predictor and Pre-processor)
#!gsutil cp ./dist/my_custom_code-0.1.tar.gz gs://{BUCKET_NAME}/custom_prediction_routine/my_custom_code-0.1.tar.gz
#  Artifacts - model + preprocess
!gsutil cp model.h5 preprocessor.pkl gs://{BUCKET_NAME}/custom_prediction_routine/model/

Copying file://model.h5 [Content-Type=application/octet-stream]...
Copying file://preprocessor.pkl [Content-Type=application/octet-stream]...
-
Operation completed over 2 objects/47.5 KiB.                                     


In [0]:
!gcloud ai-platform models create {MODEL_NAME} \
  --regions {REGION}

Created ml engine model [projects/autodeploy1-264112/models/IrisPredictor1].


# Get Prediction from Model

In [0]:
pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.6/dist-packages (1.7.11)


In [0]:
import googleapiclient.discovery

instances = [
  [6.7, 3.1, 4.7, 1.5],
  [4.6, 3.1, 1.5, 0.2],
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

['versicolor', 'setosa']
